In [1]:
import itertools
import json
import logging
import math
import multiprocessing
import random
import sys
import time
import os
from threading import Thread
from urllib.parse import unquote,quote
import aiohttp
import pandas as pd
import pymongo
import requests
from lxml import etree
from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED
import hashlib
import csv


In [2]:
client = pymongo.MongoClient(host='localhost', port=27017)
db = client['traffic']
metro_station = db['metro_station']
# metro_station.drop()
# metro_station = db['metro_station']
metro_line = db['metro_line']



In [17]:
# base_url = 'https://restapi.amap.com/v3/place/text?types=150500&city=110000&offset=50&page={p}&key=45529460f30b54520b35b4c185a4a5df&extensions=all'
# purl = base_url.format(p=1)
# session = requests.Session()
# req = session.get(purl)
# gd_json = req.json()
# gd_json

{'status': '0', 'info': 'USER_DAILY_QUERY_OVER_LIMIT', 'infocode': '10044'}

In [28]:
base_url = 'https://restapi.amap.com/v3/place/text?types=150500&city={citycode}&offset=50&page={p}&key=dfc42ab57a9fa98a7a7aecf7d30dc9c8&extensions=all'

def insert_station(citycode,pnum):
    global base_url
    retry = 0
    while True:
        try:
            purl = base_url.format(citycode=citycode,p=pnum)
            session = requests.Session()
            req = session.get(purl)
            gd_json = req.json()
            pois = gd_json['pois']
            poi_field = ['id','pcode','typecode','cityname','adname','address','adcode','name','location']
            for j in range(len(pois)):
                if pois[j]['typecode']!= '150500':
                    continue
                station = {}
#                 station['purl'] = purl
                for i in range(len(poi_field)):
                    if poi_field[i] == 'id':
                        station['_id'] = pois[j][poi_field[i]]
                    elif poi_field[i]== 'address':
                        address = pois[j][poi_field[i]].replace('(在建)','')
                        address = address.replace('(',';')
                        address = address.replace(')',';')
                        
                        address = address.split(';')
                        lines = address
                        lines = [l for l in lines if l.strip()!=''] 
                        station['metro_lines'] = lines
                    elif poi_field[i] == 'name':
                        station[poi_field[i]] = pois[j][poi_field[i]].replace('(地铁站)','')
                    elif poi_field[i] == 'location':
                        lng, lat = pois[j][poi_field[i]].split(',')
                        station['lng'] = float(lng)
                        station['lat'] = float(lat)
                    else:
                        station[poi_field[i]] = pois[j][poi_field[i]]
                try:
                    metro_station.update({'_id':station['_id']},station,upsert=True)
                except Exception as e:
                    print(e)
            break
        except Exception as e2:
            retry += 1
            if retry > 5:
                break
            print(e2)


def thread_main():
     with ThreadPoolExecutor(max_workers=10) as executor:
        future_tasks = []
        for citycode in ['110000','440300']: 
            for i in range(1,10):
                future_tasks.append(executor.submit(insert_station,citycode,i))
        wait(future_tasks, return_when=ALL_COMPLETED)
thread_main()

D:\Anaconda3\envs\tensor_env\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [35]:


def get_lines():
    for station in metro_station.find():
        cityname = station['cityname']
        lines = station['metro_lines']
        for line in lines:
            mline= {}
            id_str = '%s_%s'%(cityname,line)
            hl = hashlib.md5()
            hl.update(id_str.encode(encoding='utf-8'))
            mline['_id'] = hl.hexdigest()
            mline['name'] = line
            mline['cityname'] =cityname
    #         mline['stations']
            mline['type'] = '地铁线'
#             metro_line.update({'_id':mline['_id']},mline,upsert=True)

            metro_line.update({'_id':mline['_id']},{'$addToSet':{'stations':station['_id']}},upsert=True)


        
get_lines()
# def get_lines():
    

D:\Anaconda3\envs\tensor_env\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [3]:
metro_station2 = db['metro_station_v2']

metro_line2 = db['metro_line_v2']
metro_line2 = db['metro_line_v2']

In [18]:
ids = []
k = 1
fs = open('adjoin_stations.csv', 'w', encoding='utf_8_sig', newline='')
csv_w = csv.writer(fs)
col = [
    'pid1','no','pid2','id','name',
]
csv_w.writerow(col)

for item in metro_line2.find():
    st_l = item['st']
    for i in range(len(st_l)-1):
        csv_w.writerow([st_l[i]['poiid'],str(i+1),st_l[i + 1]['poiid'],str(k)])
        k+=1
        
fs.close()


In [16]:
ids[0]

['BV10013454', 1, 'BV11494889']

In [5]:
# gd_metro
def get_station_line_v2():
    req = requests.get(
        'http://map.amap.com/service/subway?_1641198485460&srhdata=4403_drw_shenzhen.json'
    )
    metro_li = req.json()['l']
    for line in metro_li:
        line_st = line['st']
        poi_ids = []

        for st in line_st:
            poi_ids.append({'poiid':st['poiid'],'n':st['n']})

            st['_id'] = st.pop('poiid')
            st['lng'],st['lat'] =tuple([float(s) for s in st['sl'].split(',')])
            st['cn'] = '深圳市'
            st.pop('udpx')
            st.pop('p')
            st.pop('rs')
            sr = st['r'].split('|')
            dl = []
            for r in sr:
                for l in metro_line2.find({'_id':r}):
                    dl.append({'lid':r,'ln':l['ln']})
            st['r'] = dl
            metro_station2.update({'_id': st['_id']}, st, upsert=True)
        line['st'] = poi_ids
        line['_id'] = line.pop('ls')
        line['cn'] = '深圳市'
        line.pop('c')
        line.pop('f')
        line.pop('lp')

        metro_line2.update({'_id': line['_id']}, line, upsert=True)

get_station_line_v2()

D:\Anaconda3\envs\tensor_env\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
D:\Anaconda3\envs\tensor_env\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [12]:
# pois
pois_cols = db['pois_v3']
city_code = [
    '110100', '110101', '110102', '110105', '110106', '110107', '110108',
    '110109', '110111', '110112', '110113', '110114', '110115', '110116',
    '110117', '110118', '110119'
]#北京分区的citycode
keys = [
    'ccdbc44b3f08b3d4fc6033a8ce114155', '88dedbd84615b45c7edd02baea36ef43',
    '1c012d4d0fdda4b52f1cf72b712c259f', '689eed8cd0457393991aba321719012d',
    'dfc42ab57a9fa98a7a7aecf7d30dc9c8'
]

In [10]:
for k in keys:
    get_pois(k,110108,141202)

https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=1&key=ccdbc44b3f08b3d4fc6033a8ce114155&extensions=all


D:\Anaconda3\envs\tensor_env\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=2&key=ccdbc44b3f08b3d4fc6033a8ce114155&extensions=all
https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=3&key=ccdbc44b3f08b3d4fc6033a8ce114155&extensions=all
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110108, 141202
https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=1&key=88dedbd84615b45c7edd02baea36ef43&extensions=all
https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=2&key=88dedbd84615b45c7edd02baea36ef43&extensions=all
https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=3&key=88dedbd84615b45c7edd02baea36ef43&extensions=all
finish:88dedbd84615b45c7edd02baea36ef43, 110108, 141202
https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=1&key=1c012d4d0fdda4b52f1cf72b712c259f&extensions=all
https://restapi.amap.com/v3/place/text?types=141202&city=110108&offset=50&page=2&key=1c012

In [13]:
def get_pois(key, citycode, types):
    base_url_key = 'https://restapi.amap.com/v3/place/text?types={types}&city={citycode}&offset=50&page={p}&key={key}&extensions=all'
    #     key = 'dfc42ab57a9fa98a7a7aecf7d30dc9c8'
#     citycode = '110108'
#     types = '141202'
    page = 1
    fields = [
        'type', 'typecode', 'adname', 'id', 'pcode', 'citycode', 'cityname',
        'address', 'adcode', 'business_area', 'name', 'location'
    ]
    retry = 0
    retry_pk = 0
    while True:
        try:
            purl = base_url_key.format(types=types,
                                       citycode=citycode,
                                       p=page,
                                       key=key)
#             print(purl)
            req = requests.get(purl)
            req_js = req.json()
            if 'pois' not in req_js.keys():
                retry_pk += 1
                if retry_pk > 10:
                    break
                else:
                    continue
                
            pois = req_js['pois']
            len_pois = len(pois)
#             print(len_pois)
            ids = []
            for p in pois:
                item = {}
                for field in fields:
                    if field in p.keys():
                        item[field] = p[field]
                    else:
                        item[field] = ''
                item['_id'] = item.pop('id')
                ids.append(item['_id'])
                try:
                    item['lng'], item['lat'] = tuple(
                        [float(s) for s in item['location'].split(',')])
                except Exception as e:
                    item['lng'], item['lat'] = 0, 0
                try:
                    #插入mongodb数据库
                    pois_cols.update({'_id': item['_id']}, item, upsert=True)

                except Exception as e2:
                    print('e2:', e2)
            page += 1
            if len_pois < 50:
                break
        except Exception as e3:
            print('e3', e3)
            print('e3_err: %s' % purl)
            retry += 1
            if retry > 5:
                break

    print('finish:%s, %s, %s' % (key, citycode, types))


def main():
    with ThreadPoolExecutor(max_workers=40) as executor:
        future_tasks = []
        df = pd.read_excel('./amap_poicode.xlsx')
        types_code = list(df['NEW_TYPE'])
        for citycode in city_code:
            for types in types_code:
                key = random.choice(keys)
                future_tasks.append(
                    executor.submit(get_pois, key,citycode, types))
        print(len(future_tasks))
        wait(future_tasks, return_when=ALL_COMPLETED)


In [ ]:
main()

D:\Anaconda3\envs\tensor_env\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 10102finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 10100finish:88dedbd84615b45c7edd02baea36ef43, 110100, 10108

finish:88dedbd84615b45c7edd02baea36ef43, 110100, 10109finish:689eed8cd0457393991aba321719012d, 110100, 10600finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 10000
finish:88dedbd84615b45c7edd02baea36ef43, 110100, 10103
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 10110
finish:689eed8cd0457393991aba321719012d, 110100, 10104
finish:689eed8cd0457393991aba321719012d, 110100, 10105finish:689eed8cd0457393991aba321719012d, 110100, 10300
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 10401

finish:689eed8cd0457393991aba321719012d, 110100, 10111

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 10500finish:689eed8cd0457393991aba321719012d, 110100, 10101
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 10800
finish:689eed8cd0457393991aba321719012d, 110100, 10400

finish:88dedbd84615b45c7edd02baea36ef43, 110100, 10107finish:dfc42a


finish:689eed8cd0457393991aba321719012d, 110100, 31300
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 31005


finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 31301finish:689eed8cd0457393991aba321719012d, 110100, 31000finish:689eed8cd0457393991aba321719012d, 110100, 31102

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 31500
finish:689eed8cd0457393991aba321719012d, 110100, 31401
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 31302
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 31400
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 31303
finish:689eed8cd0457393991aba321719012d, 110100, 31103
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 31501

finish:88dedbd84615b45c7edd02baea36ef43, 110100, 31800
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 31700
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 31600
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 31601finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 31702

finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 31701
finish:

finish:689eed8cd0457393991aba321719012d, 110100, 80505finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 90000

finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 80602
finish:88dedbd84615b45c7edd02baea36ef43, 110100, 80301finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 80504finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 90202
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 90200finish:689eed8cd0457393991aba321719012d, 110100, 90500
finish:88dedbd84615b45c7edd02baea36ef43, 110100, 90201



finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 90207finish:88dedbd84615b45c7edd02baea36ef43, 110100, 90203

finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 90300finish:689eed8cd0457393991aba321719012d, 110100, 90100finish:88dedbd84615b45c7edd02baea36ef43, 110100, 90205finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 90600



finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 90102finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 90206
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 90400finish:1c012

finish:689eed8cd0457393991aba321719012d, 110100, 150401
finish:689eed8cd0457393991aba321719012d, 110100, 150702

finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 150500
finish:689eed8cd0457393991aba321719012d, 110100, 150705
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 150600
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 150903finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 150403

finish:88dedbd84615b45c7edd02baea36ef43, 110100, 150100
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 150800
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110100, 150904finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 150200

finish:689eed8cd0457393991aba321719012d, 110100, 150900
finish:88dedbd84615b45c7edd02baea36ef43, 110100, 151300finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110100, 151000finish:689eed8cd0457393991aba321719012d, 110100, 151405finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 151404finish:689eed8cd0457393991aba321719012d, 110100, 151001
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110100, 15

...

finish:1c012d4d0fdda4b52f1cf72b712c259f, 110111, 200300
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 141500
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110112, 150101
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110112, 150106
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 141202finish:1c012d4d0fdda4b52f1cf72b712c259f, 110112, 150102
finish:88dedbd84615b45c7edd02baea36ef43, 110112, 150105

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 150107finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 150104

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 150200
finish:689eed8cd0457393991aba321719012d, 110112, 150201
finish:88dedbd84615b45c7edd02baea36ef43, 110112, 120200
finish:689eed8cd0457393991aba321719012d, 110112, 150206finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110112, 150205

finish:88dedbd84615b45c7edd02baea36ef43, 110112, 150203finish:88dedbd84615b45c7edd02baea36ef43, 110112, 150202

finish:88dedbd84615b45c7edd02baea36ef43, 110112, 150204
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110111,

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 160320
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 160333
finish:689eed8cd0457393991aba321719012d, 110112, 160331finish:689eed8cd0457393991aba321719012d, 110112, 160335

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 160332finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110112, 160338

finish:88dedbd84615b45c7edd02baea36ef43, 110112, 160340finish:689eed8cd0457393991aba321719012d, 110112, 160334finish:689eed8cd0457393991aba321719012d, 110112, 160339finish:88dedbd84615b45c7edd02baea36ef43, 110112, 160343finish:88dedbd84615b45c7edd02baea36ef43, 110112, 150908
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110112, 160337




finish:689eed8cd0457393991aba321719012d, 110112, 160341
finish:88dedbd84615b45c7edd02baea36ef43, 110112, 160345
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 160344finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 160342

finish:1c012d4d0fdda4b52f1cf72b712c259f, 110112, 160347
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110112,

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 10103
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 10107
finish:689eed8cd0457393991aba321719012d, 110113, 10104
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 10102
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 10105
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 10108
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 10109
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 10110
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 10111
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 10112finish:88dedbd84615b45c7edd02baea36ef43, 110113, 10200

finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110112, 991001
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110112, 190307
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 10300
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 10400
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 10800
finish:689eed8cd0457393991aba321719012d, 110113, 10401
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 10700
finish:6

finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 30506
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 30508
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 31000
finish:689eed8cd0457393991aba321719012d, 110113, 30702finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 30900finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 30701
finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 30803

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 31004finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 30800
finish:689eed8cd0457393991aba321719012d, 110113, 30802
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 30801finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 30700



finish:1c012d4d0fdda4b52f1cf72b712c259f, 110113, 31005finish:689eed8cd0457393991aba321719012d, 110113, 31100

finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 31200
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 31104finish:689eed8cd0457393991aba321719012d, 110113, 31300finish:88dedbd84615b45c7edd02baea36ef43, 110113, 31101
finish:1c012

finish:689eed8cd0457393991aba321719012d, 110112, 150909
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 60200
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60201
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 60300
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60202finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60302

finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110112, 190100
finish:dfc42ab57a9fa98a7a7aecf7d30dc9c8, 110113, 60305
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60304
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60306
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 60303
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60307
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 60400
finish:88dedbd84615b45c7edd02baea36ef43, 110113, 60308
finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60401finish:ccdbc44b3f08b3d4fc6033a8ce114155, 110113, 60402

finish:88dedbd84615b45c7edd02baea36ef43, 110113, 60102
finish:689eed8cd0457393991aba321719012d, 110113, 60403
finish:6

In [55]:
key = random.choice(keys)
key

'dfc42ab57a9fa98a7a7aecf7d30dc9c8'

In [18]:
for line_item in metro_line.find({'name':'4号线/大兴线'}):
    for sta in metro_station.find({'_id':{'$in':line_item['stations']}}):
        print(sta['name'])

国家图书馆
魏公村
人民大学
西红门
西苑
西直门
黄村西大街
天宫院
北京南站
动物园
公益西桥
灵境胡同
马家堡
义和庄
清源路
菜市口
安河桥北
北宫门
圆明园
北京大学东门
中关村
海淀黄庄
新街口
平安里
西四
宣武门
陶然亭
角门西
新宫
高米店北
高米店南
枣园
生物医药基地


In [11]:
import requests
from bs4 import BeautifulSoup
import json
"""
目标：爬取中国大陆地铁线路信息
要求：
	①获取相关城市的地铁数量
	②获取每个地铁站的名称
	③写入文档

"""

class Subway(object):
    def __init__(self):
        # 构造url
        self.url = "http://map.amap.com/subway/index.html?&1100"
        # 使用老版本请求头
        self.headers = {
            'user-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
        }
    # 获取数据
    def get_data(self):
        responses = requests.get(url=self.url, headers=self.headers)
        # 返回str字符串类型
        return responses.text
    # 解析每个城市地铁信息(地铁数量，站点)
    def parse_get_subway(self, ID, city, name):
        # 拼接地铁信息的url
        url = 'http://map.amap.com/service/subway?_1555502190153&srhdata=' + ID + '_drw_' + city + '.json'
        # 获取数据
        response = requests.get(url=url, headers=self.headers)
        # 传递一个参数接收返回的字符串类型
        html = response.text
        # 通过json.loads将json字符串类型转为python数据类型
        result = json.loads(html)
        # 循环遍历数据节点，所有地铁路线
        for node in result['l']:
            # "st"为地铁线的站点
            for start in node['st']:

                # 判断是否含有地铁分线
                # node:"l"里包含所有地铁路线  “la”为分线
                if len(node['la']) > 0:
                    # "ln"为1号线，2号线。。。  “n”为地铁站站名
                    print(name, node['ln'] + '(' + node['la'] + ')', start['n'])

                    with open('subway.json', 'a+', encoding='utf8') as f:

                        f.write(name + ',' + node['ln'] + '(' + node['la'] + ')' + ',' + start['n'] + '\n')

                else:

                    print(name, node['ln'], start['n'])

                    with open('subway.json', 'a+', encoding='utf8') as f:

                        f.write(name + ',' + node['ln'] + ',' + start['n'] + '\n')
    # 解析数据
    def parse_city_data(self, data):
        # 对数据进行编码
        data = data.encode('ISO-8859-1')
        data = data.decode('utf-8')
        soup = BeautifulSoup(data, 'lxml')

        # 获取城市信息
        res1 = soup.find_all(class_="city-list fl")[0]
        res2 = soup.find_all(class_="more-city-list")[0]
        # 遍历a标签
        for temp in res1.find_all('a'):
            # 城市ID值
            ID = temp['id']
            # 城市拼音名
            city_name = temp['cityname']
            # 城市名
            name = temp.get_text()
            self.parse_get_subway(ID, city_name, name)

        for temp in res2.find_all('a'):
            # 城市ID值
            ID = temp['id']
            # 城市拼音名
            city_name = temp['cityname']
            # 城市名
            name = temp.get_text()
            self.parse_get_subway(ID, city_name, name)


    def run(self):
        data =self.get_data()
        self.parse_city_data(data)


if __name__ == '__main__':
    subway = Subway()
    subway.run()

北京 S1线 苹果园
北京 S1线 金安桥
北京 S1线 四道桥
北京 S1线 桥户营
北京 S1线 上岸
北京 S1线 栗园庄
北京 S1线 小园
北京 S1线 石厂
北京 1号线八通线 古城
北京 1号线八通线 八角游乐园
北京 1号线八通线 八宝山
北京 1号线八通线 玉泉路
北京 1号线八通线 五棵松
北京 1号线八通线 万寿路
北京 1号线八通线 公主坟
北京 1号线八通线 军事博物馆
北京 1号线八通线 木樨地
北京 1号线八通线 南礼士路
北京 1号线八通线 复兴门
北京 1号线八通线 西单
北京 1号线八通线 天安门西
北京 1号线八通线 天安门东
北京 1号线八通线 王府井
北京 1号线八通线 东单
北京 1号线八通线 建国门
北京 1号线八通线 永安里
北京 1号线八通线 国贸
北京 1号线八通线 大望路
北京 1号线八通线 四惠
北京 1号线八通线 四惠东
北京 1号线八通线 高碑店
北京 1号线八通线 传媒大学
北京 1号线八通线 双桥
北京 1号线八通线 管庄
北京 1号线八通线 八里桥
北京 1号线八通线 通州北苑
北京 1号线八通线 果园
北京 1号线八通线 九棵树
北京 1号线八通线 梨园
北京 1号线八通线 临河里
北京 1号线八通线 土桥
北京 1号线八通线 花庄
北京 1号线八通线 环球度假区
北京 2号线 积水潭
北京 2号线 鼓楼大街
北京 2号线 安定门
北京 2号线 雍和宫
北京 2号线 东直门
北京 2号线 东四十条
北京 2号线 朝阳门
北京 2号线 建国门
北京 2号线 北京站
北京 2号线 崇文门
北京 2号线 前门
北京 2号线 和平门
北京 2号线 宣武门
北京 2号线 长椿街
北京 2号线 复兴门
北京 2号线 阜成门
北京 2号线 车公庄
北京 2号线 西直门
北京 4号线大兴线 天宫院
北京 4号线大兴线 生物医药基地
北京 4号线大兴线 义和庄
北京 4号线大兴线 黄村火车站
北京 4号线大兴线 黄村西大街
北京 4号线大兴线 清源路
北京 4号线大兴线 枣园
北京 4号线大兴线 高米店南
北京 4号线大兴线 高米店北
北京 4号线大兴线 西红门
北京 4号线大兴线 新宫
北京 4号线大兴线 公益西桥
北京 4号线大兴线 角门西
北京 4号线大兴线 马家堡
北京 4号线大兴线 北京南站
北京 4号线大

上海 7号线 刘行
上海 7号线 潘广路
上海 7号线 罗南新村
上海 7号线 美兰湖
上海 8号线 沈杜公路
上海 8号线 联航路
上海 8号线 江月路
上海 8号线 浦江镇
上海 8号线 芦恒路
上海 8号线 凌兆新村
上海 8号线 东方体育中心
上海 8号线 杨思
上海 8号线 成山路
上海 8号线 耀华路
上海 8号线 中华艺术宫
上海 8号线 西藏南路
上海 8号线 陆家浜路
上海 8号线 老西门
上海 8号线 大世界
上海 8号线 人民广场
上海 8号线 曲阜路
上海 8号线 中兴路
上海 8号线 西藏北路
上海 8号线 虹口足球场
上海 8号线 曲阳路
上海 8号线 四平路
上海 8号线 鞍山新村
上海 8号线 江浦路
上海 8号线 黄兴路
上海 8号线 延吉中路
上海 8号线 黄兴公园
上海 8号线 翔殷路
上海 8号线 嫩江路
上海 8号线 市光路
上海 9号线 曹路
上海 9号线 民雷路
上海 9号线 顾唐路
上海 9号线 金海路
上海 9号线 金吉路
上海 9号线 金桥
上海 9号线 台儿庄路
上海 9号线 蓝天路
上海 9号线 芳甸路
上海 9号线 杨高中路
上海 9号线 世纪大道
上海 9号线 商城路
上海 9号线 小南门
上海 9号线 陆家浜路
上海 9号线 马当路
上海 9号线 打浦桥
上海 9号线 嘉善路
上海 9号线 肇嘉浜路
上海 9号线 徐家汇
上海 9号线 宜山路
上海 9号线 桂林路
上海 9号线 漕河泾开发区
上海 9号线 合川路
上海 9号线 星中路
上海 9号线 七宝
上海 9号线 中春路
上海 9号线 九亭
上海 9号线 泗泾
上海 9号线 佘山
上海 9号线 洞泾
上海 9号线 松江大学城
上海 9号线 松江新城
上海 9号线 松江体育中心
上海 9号线 醉白池
上海 9号线 松江南站
上海 10号线(航中路-基隆路) 基隆路
上海 10号线(航中路-基隆路) 港城路
上海 10号线(航中路-基隆路) 高桥
上海 10号线(航中路-基隆路) 高桥西
上海 10号线(航中路-基隆路) 双江路
上海 10号线(航中路-基隆路) 国帆路
上海 10号线(航中路-基隆路) 新江湾城
上海 10号线(航中路-基隆路) 殷高东路
上海 10号线(航中路-基隆路) 三门路
上海 10号线(航中路-基隆路) 江湾体育场
上海 10

广州 9号线 花都广场
广州 9号线 花果山公园
广州 9号线 花城路
广州 9号线 广州北站
广州 9号线 花都汽车城
广州 9号线 飞鹅岭
广州 13号线 鱼珠
广州 13号线 裕丰围
广州 13号线 双岗
广州 13号线 南海神庙
广州 13号线 夏园
广州 13号线 南岗
广州 13号线 沙村
广州 13号线 白江
广州 13号线 新塘
广州 13号线 官湖
广州 13号线 新沙
广州 14号线 嘉禾望岗
广州 14号线 白云东平
广州 14号线 夏良
广州 14号线 太和
广州 14号线 竹料
广州 14号线 钟落潭
广州 14号线 马沥
广州 14号线 新和
广州 14号线 太平
广州 14号线 神岗
广州 14号线 赤草
广州 14号线 从化客运站
广州 14号线 东风
广州 14号线支线(知识城线) 新和
广州 14号线支线(知识城线) 红卫
广州 14号线支线(知识城线) 新南
广州 14号线支线(知识城线) 枫下
广州 14号线支线(知识城线) 知识城
广州 14号线支线(知识城线) 何棠下
广州 14号线支线(知识城线) 旺村
广州 14号线支线(知识城线) 汤村
广州 14号线支线(知识城线) 镇龙北
广州 14号线支线(知识城线) 镇龙
广州 18号线 冼村
广州 18号线 磨碟沙
广州 18号线 龙潭
广州 18号线 沙溪
广州 18号线 南村万博
广州 18号线 番禺广场
广州 18号线 横沥
广州 18号线 万顷沙
广州 21号线 员村
广州 21号线 天河公园
广州 21号线 棠东
广州 21号线 黄村
广州 21号线 大观南路
广州 21号线 天河智慧城
广州 21号线 神舟路
广州 21号线 科学城
广州 21号线 苏元
广州 21号线 水西
广州 21号线 长平
广州 21号线 金坑
广州 21号线 镇龙西
广州 21号线 镇龙
广州 21号线 中新
广州 21号线 坑贝
广州 21号线 凤岗
广州 21号线 朱村
广州 21号线 山田
广州 21号线 钟岗
广州 21号线 增城广场
广州 APM线 广州塔
广州 APM线 海心沙
广州 APM线 大剧院
广州 APM线 花城大道
广州 APM线 妇儿中心
广州 APM线 黄埔大道
广州 APM线 天河南
广州 APM线 体育中心南
广州 APM线 林和西
广州 广佛线 沥滘


武汉 16号线 沌口
武汉 16号线 小军山
武汉 16号线 枫林
武汉 16号线 大军山
武汉 16号线 桂子湖
武汉 16号线 马影河
武汉 16号线 协子河
武汉 16号线 湾湖
武汉 16号线 周家河
武汉 21号线(阳逻线) 后湖大道
武汉 21号线(阳逻线) 百步亭花园路
武汉 21号线(阳逻线) 新荣客运站
武汉 21号线(阳逻线) 幸福湾
武汉 21号线(阳逻线) 朱家河
武汉 21号线(阳逻线) 谌家矶
武汉 21号线(阳逻线) 青龙
武汉 21号线(阳逻线) 高车
武汉 21号线(阳逻线) 武湖
武汉 21号线(阳逻线) 沙口
武汉 21号线(阳逻线) 军民村
武汉 21号线(阳逻线) 武生院
武汉 21号线(阳逻线) 阳逻
武汉 21号线(阳逻线) 阳逻开发区
武汉 21号线(阳逻线) 施岗
武汉 21号线(阳逻线) 金台
天津 1号线 刘园
天津 1号线 瑞景新苑
天津 1号线 佳园里
天津 1号线 本溪路
天津 1号线 勤俭道
天津 1号线 洪湖里
天津 1号线 西站
天津 1号线 西北角
天津 1号线 西南角
天津 1号线 二纬路
天津 1号线 海光寺
天津 1号线 鞍山道
天津 1号线 营口道
天津 1号线 小白楼
天津 1号线 下瓦房
天津 1号线 南楼
天津 1号线 土城
天津 1号线 陈塘庄
天津 1号线 复兴门
天津 1号线 华山里
天津 1号线 财经大学
天津 1号线 双林
天津 1号线 李楼
天津 1号线 高庄子
天津 1号线 国家会展中心
天津 1号线 国瑞路
天津 1号线 东沽路
天津 2号线 滨海国际机场
天津 2号线 空港经济区
天津 2号线 国山路
天津 2号线 登州路
天津 2号线 屿东城
天津 2号线 翠阜新村
天津 2号线 靖江路
天津 2号线 顺驰桥
天津 2号线 远洋国际中心
天津 2号线 天津站
天津 2号线 建国道
天津 2号线 东南角
天津 2号线 鼓楼
天津 2号线 西南角
天津 2号线 广开四马路
天津 2号线 长虹公园
天津 2号线 咸阳路
天津 2号线 芥园西道
天津 2号线 卞兴
天津 2号线 曹庄
天津 3号线 小淀
天津 3号线 丰产河
天津 3号线 华北集团
天津 3号线 天士力
天津 3号线 宜兴埠
天津 3号线 张兴庄
天津 3号线 铁东路
天津 3号线 北站
天津 3号线 中山

重庆 6号线 长生桥
重庆 6号线 刘家坪
重庆 6号线 上新街
重庆 6号线 小什字
重庆 6号线 大剧院
重庆 6号线 江北城
重庆 6号线 五里店
重庆 6号线 红土地
重庆 6号线 黄泥塝
重庆 6号线 红旗河沟
重庆 6号线 花卉园
重庆 6号线 大龙山
重庆 6号线 冉家坝
重庆 6号线 光电园
重庆 6号线 大竹林
重庆 6号线 康庄
重庆 6号线 九曲河
重庆 6号线 礼嘉
重庆 6号线 金山寺
重庆 6号线 曹家湾
重庆 6号线 蔡家
重庆 6号线 向家岗
重庆 6号线 龙凤溪
重庆 6号线 状元碑
重庆 6号线 西南大学
重庆 6号线 北碚
重庆 6号线支线(国博线) 沙河坝
重庆 6号线支线(国博线) 红岩坪
重庆 6号线支线(国博线) 复兴
重庆 6号线支线(国博线) 思源
重庆 6号线支线(国博线) 刘家院子
重庆 6号线支线(国博线) 清溪河
重庆 6号线支线(国博线) 王家庄
重庆 6号线支线(国博线) 悦来
重庆 6号线支线(国博线) 国博中心
重庆 6号线支线(国博线) 高义口
重庆 6号线支线(国博线) 黄茅坪
重庆 6号线支线(国博线) 欢乐谷
重庆 6号线支线(国博线) 礼嘉
重庆 10号线 鲤鱼池
重庆 10号线 红土地
重庆 10号线 龙头寺公园
重庆 10号线 重庆北站南广场
重庆 10号线 重庆北站北广场
重庆 10号线 民心佳园
重庆 10号线 三亚湾
重庆 10号线 上湾路
重庆 10号线 环山公园
重庆 10号线 长河
重庆 10号线 江北机场T3航站楼
重庆 10号线 江北机场T2航站楼
重庆 10号线 渝北广场
重庆 10号线 鹿山
重庆 10号线 中央公园东
重庆 10号线 中央公园
重庆 10号线 中央公园西
重庆 10号线 悦来
重庆 10号线 王家庄
重庆 环线 重庆图书馆
重庆 环线 天星桥
重庆 环线 沙坪坝
重庆 环线 重庆大学
重庆 环线 玉带山
重庆 环线 南桥寺
重庆 环线 体育公园
重庆 环线 冉家坝
重庆 环线 动步公园
重庆 环线 洪湖东路
重庆 环线 民安大道
重庆 环线 重庆北站南广场
重庆 环线 渝鲁
重庆 环线 五里店
重庆 环线 弹子石
重庆 环线 涂山
重庆 环线 仁济
重庆 环线 上新街
重庆 环线 海棠溪
重庆 环线 罗家坝
重庆 环线 四公里
重庆 环线 南湖
重庆

成都 1号线(科学城-韦家碾) 天府五街
成都 1号线(科学城-韦家碾) 天府三街
成都 1号线(科学城-韦家碾) 世纪城
成都 1号线(科学城-韦家碾) 锦城广场
成都 1号线(科学城-韦家碾) 孵化园
成都 1号线(科学城-韦家碾) 金融城
成都 1号线(科学城-韦家碾) 高新
成都 1号线(科学城-韦家碾) 火车南站
成都 1号线(科学城-韦家碾) 桐梓林
成都 1号线(科学城-韦家碾) 倪家桥
成都 1号线(科学城-韦家碾) 省体育馆
成都 1号线(科学城-韦家碾) 华西坝
成都 1号线(科学城-韦家碾) 锦江宾馆
成都 1号线(科学城-韦家碾) 天府广场
成都 1号线(科学城-韦家碾) 骡马市
成都 1号线(科学城-韦家碾) 文殊院
成都 1号线(科学城-韦家碾) 人民北路
成都 1号线(科学城-韦家碾) 火车北站
成都 1号线(科学城-韦家碾) 升仙湖
成都 1号线(科学城-韦家碾) 韦家碾
成都 2号线 犀浦
成都 2号线 天河路
成都 2号线 百草路
成都 2号线 金周路
成都 2号线 金科北路
成都 2号线 迎宾大道
成都 2号线 茶店子客运站
成都 2号线 羊犀立交
成都 2号线 一品天下
成都 2号线 蜀汉路东
成都 2号线 白果林
成都 2号线 中医大省医院
成都 2号线 通惠门
成都 2号线 人民公园
成都 2号线 天府广场
成都 2号线 春熙路
成都 2号线 东门大桥
成都 2号线 牛王庙
成都 2号线 牛市口
成都 2号线 东大路
成都 2号线 塔子山公园
成都 2号线 成都东客站
成都 2号线 成渝立交
成都 2号线 惠王陵
成都 2号线 洪河
成都 2号线 成都行政学院
成都 2号线 大面铺
成都 2号线 连山坡
成都 2号线 界牌
成都 2号线 书房
成都 2号线 龙平路
成都 2号线 龙泉驿
成都 3号线 成都医学院
成都 3号线 石油大学
成都 3号线 钟楼
成都 3号线 马超西路
成都 3号线 团结新区
成都 3号线 锦水河
成都 3号线 三河场
成都 3号线 金华寺东路
成都 3号线 植物园
成都 3号线 军区总医院
成都 3号线 熊猫大道
成都 3号线 动物园
成都 3号线 昭觉寺南路
成都 3号线 驷马桥
成都 3号线 李家沱
成都 3号线 前锋路
成都 3号线 红星桥
成都 3号线 市二医院
成都 3号线 春熙路
成

昆明 2号线 塘子巷
昆明 2号线 东风广场
昆明 2号线 交三桥
昆明 2号线 穿心鼓楼
昆明 2号线 火车北站
昆明 2号线 白云路
昆明 2号线 金星
昆明 2号线 北辰
昆明 2号线 霖雨桥
昆明 2号线 羊肠村
昆明 2号线 司家营
昆明 2号线 龙头街
昆明 2号线 北部汽车站
昆明 3号线 西山公园
昆明 3号线 车家壁
昆明 3号线 普坪村
昆明 3号线 石咀
昆明 3号线 大渔路
昆明 3号线 西部汽车站
昆明 3号线 眠山
昆明 3号线 昌源中路
昆明 3号线 西苑
昆明 3号线 梁家河
昆明 3号线 市体育馆
昆明 3号线 潘家湾
昆明 3号线 五一路
昆明 3号线 东风广场
昆明 3号线 拓东体育馆
昆明 3号线 大树营
昆明 3号线 金马寺
昆明 3号线 太平村
昆明 3号线 虹桥
昆明 3号线 东部汽车站
昆明 4号线 金川路
昆明 4号线 大河埂
昆明 4号线 海屯路
昆明 4号线 小屯
昆明 4号线 金鼎山北路
昆明 4号线 苏家塘
昆明 4号线 小菜园
昆明 4号线 火车北站
昆明 4号线 白龙路
昆明 4号线 大树营
昆明 4号线 菊花村
昆明 4号线 菊华
昆明 4号线 和甸营
昆明 4号线 牛街庄
昆明 4号线 朱家村
昆明 4号线 羊甫头
昆明 4号线 玉缘路
昆明 4号线 广卫
昆明 4号线 塔密
昆明 4号线 斗南
昆明 4号线 金桂街
昆明 4号线 梅子村
昆明 4号线 古城
昆明 4号线 可乐村
昆明 4号线 祥丰街
昆明 4号线 牛头山
昆明 4号线 联大街
昆明 4号线 吴家营
昆明 4号线 昆明南火车站
昆明 6号线 机场中心
昆明 6号线 机场前
昆明 6号线 大板桥
昆明 6号线 东部汽车站
昆明 6号线 菊华
昆明 6号线 东郊路
昆明 6号线 岔街
昆明 6号线 塘子巷
西安 1号线 沣河森林公园
西安 1号线 北槐
西安 1号线 上林路
西安 1号线 沣东自贸园
西安 1号线 后卫寨
西安 1号线 三桥
西安 1号线 皂河
西安 1号线 枣园
西安 1号线 汉城路
西安 1号线 开远门
西安 1号线 劳动路
西安 1号线 玉祥门
西安 1号线 洒金桥
西安 1号线 北大街
西安 1号线 五路口
西安 1号线 朝阳门
西安 1号线 康复路
西安 1号线 通化门
西安 1号线 万寿路
西安 1号线 长乐坡

无锡 4号线 河埒口
无锡 4号线 西园弄
无锡 4号线 体育中心
无锡 4号线 夏家边
无锡 4号线 蠡湖大桥
无锡 4号线 大剧院
无锡 4号线 五湖大道
无锡 4号线 周新苑
无锡 4号线 市民中心
无锡 4号线 吴都路
无锡 4号线 丰润道
无锡 4号线 博览中心
青岛 1号线 东郭庄
青岛 1号线 沟岔
青岛 1号线 农业大学
青岛 1号线 正阳中路
青岛 1号线 小寨子
青岛 1号线 凤岗路
青岛 1号线 流亭
青岛 1号线 仙家寨(汽车北站)
青岛 1号线 瑞金路
青岛 1号线 遵义路
青岛 1号线 南岭
青岛 1号线 兴国路
青岛 1号线 永年路
青岛 1号线 沧安路
青岛 1号线 青岛北站
青岛 1号线 安顺路
青岛 1号线 胜利桥(纺织谷)
青岛 1号线 中心医院
青岛 1号线 水清沟
青岛 1号线 北岭
青岛 1号线 小村庄
青岛 1号线 海泊桥(海慈医疗)
青岛 1号线 台东
青岛 1号线 广饶路
青岛 1号线 观象山(市立医院)
青岛 1号线 中山路
青岛 1号线 青岛站
青岛 1号线 团岛
青岛 1号线 凤凰岛
青岛 1号线 山里
青岛 1号线 南北屯
青岛 1号线 新港山路
青岛 1号线 安子
青岛 1号线 天目山路
青岛 1号线 薛家岛
青岛 1号线 丁家河
青岛 1号线 井冈山路
青岛 1号线 太行山路
青岛 1号线 石油大学
青岛 1号线 王家港
青岛 2号线 李村公园
青岛 2号线 李村
青岛 2号线 枣山路
青岛 2号线 华楼山路
青岛 2号线 东韩
青岛 2号线 辽阳东路
青岛 2号线 同安路
青岛 2号线 苗岭路
青岛 2号线 石老人浴场
青岛 2号线 海安路
青岛 2号线 海川路
青岛 2号线 海游路
青岛 2号线 麦岛
青岛 2号线 高雄路
青岛 2号线 燕儿岛路
青岛 2号线 浮山所
青岛 2号线 五四广场
青岛 2号线 芝泉路
青岛 2号线 台东
青岛 2号线 利津路
青岛 2号线 泰山路
青岛 3号线 青岛北站
青岛 3号线 永平路
青岛 3号线 振华路
青岛 3号线 君峰路
青岛 3号线 李村
青岛 3号线 万年泉路
青岛 3号线 海尔路
青岛 3号线 地铁大厦
青岛 3号线 长沙路
青岛 3号线 双山
青岛 3号线 清江路
青岛 3号线 错埠岭
青岛 3号线 敦化路
青岛 3号线 宁夏路
青岛 3号线 江西路

哈尔滨 3号线 市第二医院
哈尔滨 3号线 海河东路
哈尔滨 3号线 会展中心
哈尔滨 3号线 湘江路
哈尔滨 3号线 珠江路
哈尔滨 3号线 公滨路
哈尔滨 3号线 油坊街
哈尔滨 3号线 汽轮机厂
哈尔滨 3号线 进乡街
哈尔滨 3号线 劳动公园
哈尔滨 3号线 旭升街
哈尔滨 3号线 肿瘤医院
哈尔滨 3号线 征仪路
哈尔滨 3号线 医大二院
哈尔滨 3号线 哈西大街
哈尔滨 3号线 凯盛源广场
哈尔滨 3号线 哈尔滨西站
哈尔滨 3号线 城乡路
哈尔滨 3号线 工农大街
哈尔滨 3号线 医大一院群力院区
哈尔滨 3号线 群力第五大道
哈尔滨 3号线 体育公园
石家庄 1号线 西王
石家庄 1号线 时光街
石家庄 1号线 长城桥
石家庄 1号线 和平医院
石家庄 1号线 烈士陵园
石家庄 1号线 新百广场
石家庄 1号线 解放广场
石家庄 1号线 平安大街
石家庄 1号线 北国商城
石家庄 1号线 博物院
石家庄 1号线 体育场
石家庄 1号线 北宋
石家庄 1号线 谈固
石家庄 1号线 朝晖桥
石家庄 1号线 白佛
石家庄 1号线 留村
石家庄 1号线 火炬广场
石家庄 1号线 石家庄东站
石家庄 1号线 南村
石家庄 1号线 洨河大道
石家庄 1号线 西庄
石家庄 1号线 东庄
石家庄 1号线 会展中心
石家庄 1号线 商务中心
石家庄 1号线 园博园
石家庄 1号线 福泽
石家庄 2号线 柳辛庄
石家庄 2号线 庄窠·铁道大学
石家庄 2号线 义堂
石家庄 2号线 建和桥
石家庄 2号线 长安公园
石家庄 2号线 北国商城
石家庄 2号线 裕华路
石家庄 2号线 槐中路
石家庄 2号线 欧韵公园
石家庄 2号线 元村
石家庄 2号线 石家庄站
石家庄 2号线 塔坛
石家庄 2号线 仓丰路留村
石家庄 2号线 南位
石家庄 2号线 嘉华路
石家庄 3号线 西三庄
石家庄 3号线 高柱
石家庄 3号线 柏林庄
石家庄 3号线 市庄
石家庄 3号线 市二中
石家庄 3号线 新百广场
石家庄 3号线 东里
石家庄 3号线 槐安桥
石家庄 3号线 西三教
石家庄 3号线 石家庄站
石家庄 3号线 汇通路
石家庄 3号线 孙村
石家庄 3号线 塔冢
石家庄 3号线 东王
石家庄 3号线 南王
石家庄 3号线 位同
石家庄 3号线 东二环南路
石家庄 3号线 西仰陵


[1]